# instalar las versiones de los paquetes que necesitamos

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install mtcnn
!pip install keras_applications==1.0.4 --no-deps
!pip install tensorflow==1.14.0
!pip install keras==2.2.4

# BAJAR una imagen de Sharon Stone:

In [ ]:
!wget https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/03/sharon_stone1.jpg
!wget https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/03/channing_tatum.jpg
!wget https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/03/sharon_stone2.jpg
!wget https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/03/sharon_stone3.jpg

In [ ]:
# Usamos MTCNN para segmentar la imagen y obtener la cara

In [ ]:
# ejemplo de 
import keras
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
 
def extract_face(filename, required_size=(224, 224)):
	pixels = pyplot.imread(filename)
	detector = MTCNN()
	# detectar caras en la imagen
	results = detector.detect_faces(pixels)
	# obtener bounding box
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# obtener cara
	face = pixels[y1:y2, x1:x2]
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array
 
# cargar y extraer cara
pixels = extract_face('sharon_stone1.jpg')
pyplot.imshow(pixels)
pyplot.show()

In [ ]:
# Ejemplo de detección de caras con el modelo vggface
from keras import applications
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

# extraer una cara de la fotografía
def extract_face(filename, required_size=(224, 224)):
	# cargar la imagen de una archivo
	pixels = pyplot.imread(filename)
	# crear un detector usando pesos estándar
	detector = MTCNN()
	# detectar la cara en la imagen
	results = detector.detect_faces(pixels)
	# extraer la bounding box
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extraer la cara
	face = pixels[y1:y2, x1:x2]
	# redimensionar los píxeles al tamaño requerido
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# cargar la foto y extraer la cara
pixels = extract_face('sharon_stone1.jpg')
# convertir la cara en un array de ejemplos
pixels = pixels.astype('float32')
samples = expand_dims(pixels, axis=0)
# preprocesar la imagen
samples = preprocess_input(samples, version=2)
# crear un modelo vggface
model = VGGFace(model='resnet50')
# predecir
yhat = model.predict(samples)
# convertir la predicción en nombres
results = decode_predictions(yhat)
# mostrar los resultados más probables
for result in results[0]:
	print('%s: %.3f%%' % (result[0], result[1]*100))

In [ ]:
# verificación con el modelo VGGFace2
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

# extraer las caras y obtener los embeddings para una lista de archivos
def get_embeddings(filenames):
	# extraer caras
	faces = [extract_face(f) for f in filenames]
	# convertir en array de ejemplos
	samples = asarray(faces, 'float32')
	# preprocesar
	samples = preprocess_input(samples, version=2)
	# crear el modelo sin el TOP
	model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	# predecir los embeddings
	yhat = model.predict(samples)
	return yhat

# determinar si el candidato es un match para para la cara conocida
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calcular la distancia entre embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		print('>la cara es un Match (%.3f <= %.3f)' % (score, thresh))
	else:
		print('>la cara NO es un Match (%.3f > %.3f)' % (score, thresh))

# definir archivos
filenames = ['sharon_stone1.jpg', 'sharon_stone2.jpg',
	'sharon_stone3.jpg', 'channing_tatum.jpg']
# obtener embeddings de las imágenes
embeddings = get_embeddings(filenames)
# definir la identidad de sharon stone
sharon_id = embeddings[0]
# verificar fotos de sharon stone
print('Positive Tests')
is_match(embeddings[0], embeddings[1])
is_match(embeddings[0], embeddings[2])
# verificar que no hay un match con otras personas
print('Negative Tests')
is_match(embeddings[0], embeddings[3])